In [1]:
# Import necessary packages
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from aif360.algorithms.postprocessing import (CalibratedEqOddsPostprocessing, EqOddsPostprocessing,RejectOptionClassification)
from aif360.datasets import StandardDataset
from torchvision import models,transforms
import pickle
from collections import OrderedDict

pip install 'aif360[AdversarialDebiasing]'


In [2]:
# load resnet50 model and modify it to match the one from the github to load the weights from the pkl
# resnet50 trained on VGGFace2
resnet50 = models.resnet50(pretrained=False)
print(resnet50.maxpool)
resnet50.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
print(resnet50.maxpool)
print(resnet50.layer2[0].conv1)
resnet50.layer2[0].conv1 = nn.Conv2d(256, 128, kernel_size=(1, 1), stride=(2, 2), bias=False)
print(resnet50.layer2[0].conv1)
print(resnet50.layer2[0].conv2)
resnet50.layer2[0].conv2 = nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
print(resnet50.layer2[0].conv2)
print(resnet50.layer3[0].conv1)
resnet50.layer3[0].conv1 = nn.Conv2d(512, 256, kernel_size=(1, 1), stride=(2, 2), bias=False)
print(resnet50.layer3[0].conv1)
print(resnet50.layer3[0].conv2)
resnet50.layer3[0].conv2 = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
print(resnet50.layer3[0].conv2)
print(resnet50.layer4[0].conv1)
resnet50.layer4[0].conv1 = nn.Conv2d(1024, 512, kernel_size=(1, 1), stride=(2, 2), bias=False)
print(resnet50.layer4[0].conv1)
print(resnet50.layer4[0].conv2)
resnet50.layer4[0].conv2 = nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
print(resnet50.layer4[0].conv2)
print(resnet50.avgpool)
resnet50.avgpool = nn.AvgPool2d(kernel_size=7, stride=1, padding=0)
print(resnet50.avgpool)
resnet50.fc = nn.Linear(in_features=2048,out_features=8631)

MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
Conv2d(256, 128, kernel_size=(1, 1), stride=(2, 2), bias=False)
Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
Conv2d(512, 256, kernel_size=(1, 1), stride=(2, 2), bias=False)
Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
Conv2d(1024, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
Conv2d(1024, 512, kernel_size=(1, 1), stride=(2, 2), bias=False)
Conv2d(512, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1

In [3]:
# load weights from pickle and load state dict into model
with open("weights/resnet50_scratch_weight.pkl", 'rb') as f:
        weights = pickle.load(f, encoding='latin1')
def f(x):
    return torch.from_numpy(x)
weights = dict(map(lambda x: (x[0], f(x[1])), weights.items()))
weights = OrderedDict(weights)
resnet50.load_state_dict(weights)

<All keys matched successfully>